### Phi-4 14B GRPO RL Fine-Tuning GSM8K

### Installation

In [1]:
import os

In [2]:
filename = 'sentinel_incident_severity_training_samples.jsonl'
import json
results = []
with open(filename, 'r') as file:
    for line in file:
        line = line.strip() # Remove any extra whitespace or newline characters
        if line: # Make sure the line isn't empty
                json_obj = json.loads(line)
                results.append(json_obj)

### Unsloth

Load up `Phi-4 14B`, and set parameters

In [4]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
max_seq_length = 512 # Can increase for longer reasoning traces
lora_rank = 16 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-4",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 03-31 05:42:25 [__init__.py:239] Automatically detected platform cuda.
Unsloth: Switching from Unsloth dynamic quant to normal quant since
we do not yet support fast inference for unsloth/phi-4-unsloth-bnb-4bit
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3. vLLM: 0.8.2.
   \\   /|    NVIDIA GeForce RTX 4090 Laptop GPU. Num GPUs = 1. Max memory: 15.992 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Your GPU cannot handle sequence lengths of 256 due to limited GPU memory.
Unsloth: Your GPU can only handle approximately the maximum sequence

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 03-31 05:42:48 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 03-31 05:42:48 [model_runner.py:1146] Model loading took 8.6253 GB and 12.526057 seconds
INFO 03-31 05:42:51 [worker.py:267] Memory profiling takes 2.30 seconds
INFO 03-31 05:42:51 [worker.py:267] the current vLLM instance can use total_gpu_memory (15.99GiB) x gpu_memory_utilization (0.61) = 9.82GiB
INFO 03-31 05:42:51 [worker.py:267] model weights take 8.63GiB; non_torch_memory takes 0.24GiB; PyTorch activation peak memory takes 0.47GiB; the rest of the memory reserved for KV Cache is 0.48GiB.
INFO 03-31 05:42:51 [executor_base.py:111] # cuda blocks: 158, # CPU blocks: 983
INFO 03-31 05:42:51 [executor_base.py:116] Maximum concurrency for 256 tokens per request: 9.88x
INFO 03-31 05:42:51 [model_runner.py:1442] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-o

Capturing CUDA graph shapes: 100%|███████████████████████████████████████████████████████████████| 19/19 [00:15<00:00,  1.21it/s]

INFO 03-31 05:43:07 [model_runner.py:1570] Graph capturing finished in 17 secs, took 4.43 GiB
INFO 03-31 05:43:07 [llm_engine.py:447] init engine (profile, create kv cache, warmup model) took 18.89 seconds



Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.3.19 patched 40 layers with 0 QKV layers, 0 O layers and 40 MLP layers.


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [1]:
import re
from sentinel_api_grader import extract_api_call_info, sentinel_api_reward_func, incremental_json_reward_func, severity_reward_func

In [60]:
SENTINEL_SYSTEM_PROMPT = (
"You are an Azure Sentinel API assistant. Return JSON exactly matching the following Sentinel API call: "
'{"method":"PUT","url":"https://management.azure.com/subscriptions/resourceGroups/'
'providers/Microsoft.OperationalInsights/workspaces/providers/Microsoft.SecurityInsights/Incidents/?api-version=2025-03-01",'
'"headers":{"Content-Type":"application/json","Authorization":"Bearer "},'
'"body":{"properties":{"severity":"<High|Medium|Low>"}}}.'
" Replace placeholders and output only JSON."
)

def truncate(text: str, max_length: int = 40) -> str:
    """Return text truncated to max_length characters (if necessary)."""
    if len(text) > max_length:
        return text[:max_length] + "..."
    return text

def format_sentinel_prompt(item: dict) -> list[dict]:
    """
    Converts a dataset item (with keys 'question' and 'answer') into a list of messages
    that includes:
    - A system message containing the Sentinel API instructions.
    - A user message that provides additional incident details extracted from the 'question' field.
    
    The dataset item is assumed to be a dictionary such as:  
      {  
         "question": { ... },  
         "answer": { ... }  
      }  
    """  
    q = item.get("question", {})
    properties    = q.get("properties", {})   
    additional_data    = properties.get("additionalData", {})   

    # Extract and truncate key incident fields.  
    incident_id   = truncate(q.get("id", "unknown ID"))  
    incident_name = truncate(q.get("name", "unknown name"))  
    title         = truncate(properties.get("title", "unknown title"))  
    alerts_count    = additional_data.get("alertsCount", "unkown alerts count")   
    tactics    = additional_data.get("tactics", "unkown tactic")  
    alert_product_names    = additional_data.get("alertProductNames", "unkown product names")  
    status        = truncate(properties.get("status", "unknown status"))  
      
    # Build a user prompt that describes the task with specific incident details.  
    user_prompt = (  
        f"Update the incident using the details below:\n"  
        f"  - Incident Name: {incident_name}\n"  
        f"  - Title: {title}\n"  
        f"  - Status: {status}\n\n"  
        f"  - AlertsCount: {alerts_count}\n\n"
        f"  - Tactics: {tactics}\n\n"
        f"  - Alert Product Names: {alert_product_names}\n\n"
        "Generate a valid API call (in JSON) to update the incident's severity using the Sentinel Incidents API."  
    )  
      
    return [  
        {"role": "system", "content": SENTINEL_SYSTEM_PROMPT},  
        {"role": "user",   "content": user_prompt}  
    ]  

def get_formatted_sentinel_data(data: list[dict]) -> list[dict]:
    """
    Given a list of dataset items (each parsed from your jsonl file with keys 'question' and 'answer'),
    returns a list where each element is a dict with:
    - 'prompt': a list of chat messages (with system and user roles)
    - 'answer': the expected API call (converted to a JSON string)
    """
    formatted_examples = []
    for item in data:
        prompt = format_sentinel_prompt(item)
        # Convert the expected answer (a dict) into a JSON string.
        answer_obj = item.get("answer", {})
        answer_str = json.dumps(answer_obj)
        formatted_examples.append({
        "prompt": prompt,
        "answer": answer_str
        })
    return formatted_examples

In [61]:
dataset = get_formatted_sentinel_data(results)

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [62]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 500,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 6


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        sentinel_api_reward_func, incremental_json_reward_func
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

<a name="Inference"></a>
### Inference

Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Update the incident using the details below:\n  - Incident Name: c9d5b4c5-49b9-412f-92dc-b63cfcebd328\n  - Title: Unfamiliar sign-in properties involving ...\n  - Status: Closed\n\n  - AlertsCount: 1\n\n  - Tactics: ['InitialAccess']\n\n  - Alert Product Names: ['Azure Active Directory Identity Protection']\n\nGenerate a valid API call (in JSON) to update the incident's severity using the Sentinel Incidents API."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

In [38]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Update the incident using the details below:\n  - Incident Name: c9d5b4c5-49b9-412f-92dc-b63cfcebd328\n  - Title: Unfamiliar sign-in properties involving ...\n  - Status: Closed\n\n  - AlertsCount: 1\n\n  - Tactics: ['InitialAccess']\n\n  - Alert Product Names: ['Azure Active Directory Identity Protection']\n\nGenerate a valid API call (in JSON) to update the incident's severity using the Sentinel Incidents API."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|█████████████████████| 1/1 [00:05<00:00,  5.80s/it, est. speed input: 19.66 toks/s, output: 24.66 toks/s]


'To update the severity of an incident using the Sentinel Incidents API, you need to use the appropriate API endpoint and provide the necessary details in JSON format. Below is an example of how you can structure your API call to update the severity of the incident with the given details:\n\n```json\n{\n  "resourceId": "<YOUR_WORKSPACE_ID>",\n  "incidentId": "c9d5b4c5-49b9-412f-92dc-b63cfcebd328",\n  "properties": {\n    "severity": "High"\n  }\n}\n```\n\n### Explanation:\n- **resourceId**: Replace `<YOUR_WORKSPACE_ID>` with the actual workspace ID where your Sentinel workspace is'

In [29]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Update the incident using the details below:\n  - Incident ID: /subscriptions/ab48f397-fc82-4634-aa52-6...\n  - Incident Name: 3ac3dbb8-9438-47a6-9fce-d1273ae79993\n  - Title: ef0b1804-ac3c-403a-924a-3d1defe282fe_ale...\n  - Status: New\n\nGenerate a valid API call (in JSON) to update the incident's severity. Ensure your output follows the exact API call template provided in the system message."},
], tokenize = False, add_generation_prompt = True)

if not text.endswith(tokenizer.eos_token):
    text += tokenizer.eos_token

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.2,
    top_p = 0.95,
    max_tokens = 4096,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|█████████████████████| 1/1 [00:04<00:00,  4.41s/it, est. speed input: 30.59 toks/s, output: 27.64 toks/s]


'<|im_start|>assistant<|im_sep|>To update the severity of an incident in a system, you typically need to make an API call or execute a command that modifies the incident\'s details. Below is a generic example of how you might structure an API call to update the severity of an incident. This example assumes you are using a RESTful API and have the necessary permissions and authentication details.\n\n### Example API Call\n\n```http\nPUT /api/v1/incidents/{incident_id}\nAuthorization: Bearer YOUR_ACCESS_TOKEN\nContent-Type: application/json\n\n{\n  "severity": "High"\n}\n```\n\n### Explanation\n\n-'

In [30]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Update the incident using the details below:\n  - Incident ID: /subscriptions/ab48f397-fc82-4634-aa52-6...\n  - Incident Name: 3ac3dbb8-9438-47a6-9fce-d1273ae79993\n  - Title: ef0b1804-ac3c-403a-924a-3d1defe282fe_ale...\n  - Status: New\n\nGenerate a valid API call (in JSON) to update the incident's severity. Ensure your output follows the exact API call template provided in the system message."},
], tokenize = False, add_generation_prompt = True)

if not text.endswith(tokenizer.eos_token):
    text += tokenizer.eos_token

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.2,
    top_p = 0.95,
    max_tokens = 4096,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|█████████████████████| 1/1 [00:05<00:00,  5.00s/it, est. speed input: 26.99 toks/s, output: 24.39 toks/s]


"<|im_start|>assistant<|im_sep|>To update an incident's severity using an API, you typically need to make an HTTP request to the appropriate endpoint provided by the incident management system you are using. Below is a generic example of how you might structure such an API call using JSON and a common HTTP method like `PATCH` or `PUT`. Note that the exact details will depend on the specific API you are working with, so you should refer to its documentation for precise details.\n\n### Example API Call\n\nAssuming you are using a RESTful API, here is a sample JSON payload and HTTP request to update an incident's severity"

In [22]:
print(type(model))

<class 'peft.peft_model.PeftModelForCausalLM'>


And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Which is bigger? 9.11 or 9.9?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

In [ ]:
print(output)

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!